In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Importing required libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import linalg
import math 
import pickle as pk
import warnings
warnings.filterwarnings("ignore")

In [5]:
# 3.b.

np.random.seed(42)

def GetData(Xm,Xc,Ym,Yc,points):
  D1 , D2 = np.random.multivariate_normal(mean = Xm, cov = Xc , size = points).T
  W1 = np.vstack((D1,D2)).T

  W2 = np.random.normal(Ym,Yc , size = points).T

  return W1,W2 

points = 100
Xm = [0, 0]
Xc = [[1, 0.5], [0.5, 1]]
Ym = 0
Yc = 0.1

# X , Y = GetData(Xm,Xc,Ym,Yc,points)


# Saving and loading data 

path = '/content/drive/MyDrive/Projects/SML_Assignment/Assignment_3/Saved_Data/'
# pk.dump(X, open(path+'X_QC.pkl', "wb"))
# pk.dump(Y, open(path+'Y_QC.pkl', "wb"))

X = pk.load(open(path+'X_QC.pkl', "rb"))
Y = pk.load(open(path+'Y_QC.pkl', "rb"))


In [6]:
X_train = X[:50]
X_test = X[50:]
Y_train = Y[:50]
Y_test = Y[50:]
print('X_train Shape : ',X_train.shape)
print('Y_train Shape : ',Y_train.shape)
print('X_test Shape : ',X_test.shape)
print('Y_test Shape : ',Y_test.shape)


X_train Shape :  (50, 2)
Y_train Shape :  (50,)
X_test Shape :  (50, 2)
Y_test Shape :  (50,)


In [7]:
# 3.c. 

# Assuming B* = B4

np.random.seed(42)

def sig(x):
  t = 1/(1+math.exp(-x))
  return t

def FeedFoward(x,B0,B1,B2,B3,B4):
  a1 = B1*x[0] + B2*x[1] * B0
  z1 = sig(a1)
  a2 = B3*z1 + B4
  z2 = sig(a2)
  y_pred = z2
  return y_pred , z1 , z2

def BackProp(x,y,B3,z1,z2):
  dB0 = -2*(y-z2)*z2*(1-z2)*z1*(1-z1)*B3
  dB1 = -2*(y-z2)*z2*(1-z2)*z1*(1-z1)*B3*x[0]
  dB2 = -2*(y-z2)*z2*(1-z2)*z1*(1-z1)*B3*x[1]
  dB3 = -2*(y-z2)*z2*(1-z2)*z1
  dB4 = -2*(y-z2)*z2*(1-z2)
  return dB0,dB1,dB2,dB3,dB4

def FitNeuralNetwork(X,Y,epochs):

  alpha = 0.3
  B0 = np.random.uniform(0.1,1)
  B1 = np.random.uniform(0.1,1)
  B2 = np.random.uniform(0.1,1)
  B3 = np.random.uniform(0.1,1)
  B4 = np.random.uniform(0.1,1)

  for i in range(epochs):
    for j in range(len(X)):
      x = X[j] 
      y = Y[j]
      ypred , z1 , z2 = FeedFoward(x,B0,B1,B2,B3,B4)
      dB0,dB1,dB2,dB3,dB4 = BackProp(x,y,B3,z1,z2)
      B0 = B0 - alpha*dB0
      B1 = B1 - alpha*dB1
      B2 = B2 - alpha*dB2
      B3 = B3 - alpha*dB3
      B4 = B4 - alpha*dB4

  return B0,B1,B2,B3,B4 

B0,B1,B2,B3,B4 = FitNeuralNetwork(X_train,Y_train,epochs=5)    
print('Weights after 5 epochs : ')
print('B0 : ',B0)
print('B1 : ',B1)
print('B2 : ',B2)
print('B3 : ',B3)
print('B* : ',B4)
  

Weights after 5 epochs : 
B0 :  0.4613730920050851
B1 :  0.9476494475914826
B2 :  0.7778893055008189
B3 :  -0.7210803819725828
B* :  -2.329168048832749


In [10]:
# 3.d.

def PredictNeuralNetwork(X,B0,B1,B2,B3,B4):
  Y_pred = []
  for j in range(len(X)):
    x = X[j] 
    ypred , z1 , z2 = FeedFoward(x,B0,B1,B2,B3,B4)
    Y_pred.append(ypred)
  return np.array(Y_pred)  

Y_pred = PredictNeuralNetwork(X_test,B0,B1,B2,B3,B4) 
print('5 predicted Values from the X_test :',Y_pred[:5])

5 predicted Values from the X_test : [0.05015262 0.05739577 0.06290489 0.0830178  0.06652001]


In [9]:
# 3.e.

def GetMSE(X,Y):
  MSE = X-Y
  MSE = np.square(MSE)
  MSE = np.mean(np.mean(MSE))
  return MSE

print('MSE for the test set : ',GetMSE(Y_test,Y_pred))  


MSE for the test set :  0.015534213954228344
